In [1]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.constraints import max_norm
from PIL import Image
import skimage
import pandas as pd
from spectrum import *

C:\Users\Sai Swaroop\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#path = '/home/grads/n/narendra5/Desktop/Programs/LER_machine_learning/'
#path = r"C:\\Users\\narendra\\Documents\\LER\\LER_machine_learning\\"
#model = load_model(path + 'models/' + 'EDGEnet_run_epoch_4.h5')


path_models = 'C:/Users/Sai Swaroop/sai_linescan/LER_machine_learning/'
path_noisy = 'C:/courses/DirectedStudies/Datasets/noisy_images/';
path = 'C:/courses/DirectedStudies/Datasets/original_images/';
path_linescan ='C:/courses/DirectedStudies/Datasets/linescans/'; 

model = load_model(path_models + 'models/' + 'EDGEnet2_int_L1_epoch_4.h5')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1024, 64, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024, 64, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024, 64, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1024, 64, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024, 64, 64)      256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024, 64, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1024, 64, 64)      36928     
__________

C:\Users\Sai Swaroop\Anaconda3\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
sigmas = [1.6]#, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8];
alphas = [0.1]#, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9];	

widths = [30]#, 30];
noises = [2, 3, 4, 5, 10, 20, 30, 50, 100, 200];

#df = pd.DataFrame(columns = ['filename_orignal','filename_noisy','image_orig_ler','image_noisy_ler'])

df = pd.DataFrame(columns = ['filename','noise', 'sigma', 'alpha', 'Xi', 'width', 'space', \
                             'psd_0f_orig','psd_0f_predict','mse_completeF','error_cF','lf_mse','error_lf', 'mf_mse','error_mf','hf_mse','error_hf'])

NFFT_VAL = 1024
range_psd = int(NFFT_VAL/2)
spectrum_data = {}
spectrum_data_im_orig = {}
spectrum_data_im_noisy = {}
spectrum_data_im_predict = {}


def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx],idx


Xis = [10];
path_noisy = 'C:/courses/DirectedStudies/Datasets/noisy_images/';
path = 'C:/courses/DirectedStudies/Datasets/original_images/';
path_linescan ='C:/courses/DirectedStudies/Datasets/linescans/'; 
count = 0;
for sigma in sigmas:
    for alpha in alphas:
        for Xi in Xis:
            for width in widths:
                for s in [0,1]:
                    for noise in noises:
                        space = math.floor(width*2**s)
                        shift = math.floor(-25 + (width + space/2 + Xi + alpha*10 + sigma*10)%16)
                        # noisy images load
                        noisy_file_name ='nim_' +  "{0:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '_' + str(noise) + '.tiff';
                        noisy_file = path_noisy + 'noisy_images/nim_' + "{0:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '_' + str(noise) + '.tiff';


                        
                        
                        # original images load
                        original_file_name = 'oim_'+ "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '.tiff'
                        original_file = path + 'original_images/oim_' + "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '.tiff'
                        
                        # linescan files load 
                        linescan_file_name = 'linescan_'+ "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '.txt'
                        linescan_path = path_linescan + 'linescans/linescan_'+ "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '.txt'
                        linescan_round_path = path_linescan + 'linescans_round/linescan_'+ "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '.txt'
                        linescan_int_path = path_linescan + 'linescans_int/linescan_'+ "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '.txt'
                          
                        
                        imnoisy = np.array(Image.open(noisy_file))
                        imoriginal = np.array(Image.open(original_file))
                        print('Validation_count: ',count)

                        
                        #fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))
                        #ax[0].imshow(imnoisy,cmap = 'gray',aspect= 0.2)
                        #ax[1].imshow(imoriginal,cmap = 'gray',aspect= 0.2)
                        #plt.show()
                        
                        linescan = []            
                        with open(linescan_path,'r') as f:
                            for i, line in enumerate(f):
                                if i < 3000:
                                    a, b = line.split(',')
                                    linescan.append(float(b))
                        
                        
                        linescan_round = []            
                        with open(linescan_round_path,'r') as f:
                            for i, line in enumerate(f):
                                if i < 3000:
                                    a, b = line.split(',')
                                    linescan_round.append(float(b))
                        
                        linescan = linescan[:2048]
                        leftline = np.array(linescan[:1024]) 
                        rightline = linescan[1024:]
                        rightline.reverse()
                        rightline = np.array(rightline)
                        leftline = leftline + shift
                        rightline = rightline + shift
                        
                        linescan_round = linescan_round[:2048]
                        leftline_round = np.array(linescan_round[:1024]) 
                        rightline_round = linescan_round[1024:]
                        rightline_round.reverse()
                        rightline_round = np.array(rightline_round)
                        leftline_round = leftline_round + shift
                        rightline_round = rightline_round + shift
                        
                        imnoisy = (imnoisy)/256
                        imnoisy = imnoisy.reshape(1,1024,64,1)
                        linepredict = model.predict(imnoisy)
                        imnoisy = imnoisy.reshape(1024,64)
                        linepredict = linepredict.reshape(1024,2)
                        linepredict_int = np.array([(x*64).round().astype(int) for x in linepredict])
                        
                        axis = np.arange(0,range_psd*2,2)
                        
                        mult_tape_spectrum = pmtm(leftline.astype(int),NFFT= NFFT_VAL,NW=4, k=6,method ='adapt', show = False)                                                                  
                        spectrum_data[linescan_file_name] = mult_tape_spectrum 
                        Sk = abs(mult_tape_spectrum[0])**2
                        Sk = np.mean(Sk * np.transpose(mult_tape_spectrum[1]), axis=0)
                        #fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 8), sharex = True, sharey=True)
                        #ax[0].loglog(axis,Sk[0:range_psd], 'bo')

                        
                        mult_tape_spectrum_predict = pmtm(linepredict_int[:,0].astype(int),NFFT= NFFT_VAL,NW=4, k=6,method ='adapt', show = False)                                                                  
                        spectrum_data_im_predict[linescan_file_name] = mult_tape_spectrum_predict 
                        Sk_predict = abs(mult_tape_spectrum_predict[0])**2
                        Sk_predict = np.mean(Sk_predict * np.transpose(mult_tape_spectrum_predict[1]), axis=0)
                        #fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 8), sharex = True, sharey=True)
                        #ax[0].loglog(axis,Sk_predict[0:range_psd], 'ro')
                        #plt.show()
                        
                        # LF < 5um-1
                        range_lf = 10 # capturing only low frequencies  (x/1024)*(1/2nm) = (5um-1) #find x
                        mse_predict_lf = ((Sk[0:range_lf] - Sk_predict[0:range_lf]) ** 2).mean()
                        error_percent_lf = mse_predict_lf/Sk[0]
                        
                        
                        
                        # mF < 50um-1 , > 5um-1
                        range_mf = 100 # capturing only mid frequencies  (x/1024)*(1/2nm) = (50um-1) #find x
                        mse_predict_mf = ((Sk[range_lf:range_mf] - Sk_predict[range_lf:range_mf]) ** 2).mean()
                        error_percent_mf = mse_predict_mf/Sk[range_lf]

                                                
                        # HF > 50um-1
                        range_hf = 100 # capturing only high frequencies 
                        mse_predict_hf = ((Sk[range_hf:512] - Sk_predict[range_hf:512]) ** 2).mean()
                        error_percent_hf = mse_predict_hf/max(Sk[range_hf:512])
                        
                        mse_complete = ((Sk - Sk_predict) ** 2).mean()
                        error_percent_cf = mse_complete/Sk[0]
                        
                        # Dumping into dataframe
                        
                        df.loc[count,'filename'] = noisy_file_name
                        df.loc[count,'noise'] = noise
                        df.loc[count,'sigma'] = sigma
                        df.loc[count,'alpha'] = alpha
                        df.loc[count,'Xi'] = Xi
                        df.loc[count,'width'] = width
                        df.loc[count,'space'] = space
                        
                        df.loc[count,'lf_mse'] = mse_predict_lf
                        df.loc[count,'mf_mse'] = mse_predict_mf
                        df.loc[count,'hf_mse'] = mse_predict_hf
                        df.loc[count,'mse_completeF'] = error_percent_cf
                        df.loc[count,'psd_0f_orig'] = Sk[0]
                        df.loc[count,'psd_0f_predict'] = Sk_predict[0]
                        
                        df.loc[count,'error_lf'] = error_percent_lf
                        df.loc[count,'error_mf'] = error_percent_mf
                        df.loc[count,'error_hf'] = error_percent_hf
                        df.loc[count,'error_cF'] = error_percent_cf
 
                        
                        count = count  + 1
                        
                        

Validation_count:  0


C:\Users\Sai Swaroop\Anaconda3\lib\site-packages\spectrum\mtm.py:529: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  complex_result = (np.issubdtype(in1.dtype, np.complex) or
C:\Users\Sai Swaroop\Anaconda3\lib\site-packages\spectrum\mtm.py:530: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  np.issubdtype(in2.dtype, np.complex))


Validation_count:  1
Validation_count:  2
Validation_count:  3
Validation_count:  4
Validation_count:  5
Validation_count:  6
Validation_count:  7
Validation_count:  8
Validation_count:  9
Validation_count:  10
Validation_count:  11
Validation_count:  12
Validation_count:  13
Validation_count:  14
Validation_count:  15
Validation_count:  16
Validation_count:  17
Validation_count:  18
Validation_count:  19


In [4]:
print(df)

                                   filename noise sigma alpha  Xi width space  \
0     nim_1.6e-09_0.1_1e-08_30_30_17_2.tiff     2   1.6   0.1  10    30    30   
1     nim_1.6e-09_0.1_1e-08_30_30_17_3.tiff     3   1.6   0.1  10    30    30   
2     nim_1.6e-09_0.1_1e-08_30_30_17_4.tiff     4   1.6   0.1  10    30    30   
3     nim_1.6e-09_0.1_1e-08_30_30_17_5.tiff     5   1.6   0.1  10    30    30   
4    nim_1.6e-09_0.1_1e-08_30_30_17_10.tiff    10   1.6   0.1  10    30    30   
5    nim_1.6e-09_0.1_1e-08_30_30_17_20.tiff    20   1.6   0.1  10    30    30   
6    nim_1.6e-09_0.1_1e-08_30_30_17_30.tiff    30   1.6   0.1  10    30    30   
7    nim_1.6e-09_0.1_1e-08_30_30_17_50.tiff    50   1.6   0.1  10    30    30   
8   nim_1.6e-09_0.1_1e-08_30_30_17_100.tiff   100   1.6   0.1  10    30    30   
9   nim_1.6e-09_0.1_1e-08_30_30_17_200.tiff   200   1.6   0.1  10    30    30   
10    nim_1.6e-09_0.1_1e-08_30_60_18_2.tiff     2   1.6   0.1  10    30    60   
11    nim_1.6e-09_0.1_1e-08_

In [5]:
df.to_csv('spectrum_comparison.csv')



## entire psd also record